### ■ 유통 판매량 예측 및 재고 최적화
# 단계 3: 모델 최적화와 비즈니스 평가

<img src = "https://github.com/Jangrae/img/blob/master/store.png?raw=true" width=800, align="left"/>

# 수행 과제

다음과 같은 과정으로 프로젝트를 진행합니다.

#### 1. 환경 설정
- 이후 진행에 필요한 환경 설정을 수행합니다.

#### 2. 4차 모델링
- Random Forest, LigntGBM 알고리즘으로 모델링합니다.
- 이번에는 하이퍼파라미터 튜닝을 통해 모델의 성능을 높여봅니다.
- 변화되는 모델의 성능을 기록합니다.

#### 3. 파이프라인 구축
- 새로 읽어온 데이터에 대해 모델이 예측핳 수 있는 형태의 데이터 셋을 만들어야 합니다.
- 이러한 데이터 셋을 만드는 파이프라인을 함수를 만들어 보세요.

#### 4. 최종 평가
- 새로운 데이터에 대해 예측하고 성능을 평가합니다.
- 재고량을 평가하는 다음 함수를 활용해 비즈니스 관점의 평가를 수행하세요.
- 최종 평가 결과를 기록하고 비교합니다.

# 1. 환경 설정

- 이후 진행에 필요한 환경 설정을 수행합니다.

## (1) 경로 설정

- 프로젝트 수행 환경에 맞게 파일 경로를 설정합니다.

### 1) 로컬 수행(Anaconda)
- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [7]:
# 기본 경로
path = ''

### 2) 구글 콜랩 수행

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [9]:
# # 구글 드라이브 연결, 패스 지정
# import sys
# if 'google.colab' in sys.modules:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     path = '/content/drive/MyDrive/project/'

## (2) 라이브러리 불러오기

- 이후 사용할 라이브러리를 모두 불러옵니다.

In [11]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.metrics import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

- 시각화에 사용할 폰트 크기를 설정합니다.

In [13]:
# 시각화 폰트 사이즈 설정
plt.rc('font', size=7)
plt.rc('xtick', labelsize=7)
plt.rc('ytick', labelsize=7)
plt.rc('axes', titlesize=12)
plt.rc('axes', labelsize=8)
plt.rc('legend', fontsize=8)
plt.rc('axes', linewidth=0.3)

- 모델이 예측한 결과를 시각화할 때 다음 함수를 활용합니다.

In [15]:
# 함수 만들기
def plot_model_result(y_train, y_val, y_pred, title=None):
    y_train = pd.Series(y_train)
    y_val = pd.Series(y_val)
    y_val.index = range(len(y_train), len(y_train) + len(y_val))
    y_pred = pd.Series(y_pred.reshape(-1,), index=y_val.index)

    # 시각화
    plt.figure(figsize=(12, 5))
    plt.subplot(2, 1, 1)
    plt.plot(y_train, label='y_train', color='tab:blue', linewidth=0.8)
    plt.plot(y_val, label='y_val', color='tab:green', linewidth=0.8)
    plt.plot(y_pred, label='y_pred', color='tab:orange', linewidth=0.8)
    plt.title(f'{title}', size=15, pad=20)
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(y_val, label = 'y_val', color='tab:green',  marker='o', markersize=2, linewidth=0.8)
    plt.plot(y_pred, label = 'y_pred', color='tab:orange',  marker='o', markersize=2, linewidth=0.8)
    plt.legend()
    plt.tight_layout()
    plt.show()

## (3) 데이터 불러오기

- 이전 과정에서 저장한 pkl 파일을 불러옵니다.
- 데이터프레임 이름은 data03, data07, data12가 되게 합니다.
- 다음과 같은 형태의 구문으로 불러옵니다.
~~~
mydata = joblib.load(path + 'mydata.pkl')
~~~

In [17]:
data03 = joblib.load('data03.pkl')
data07 = joblib.load('data07.pkl')
data12 = joblib.load('data12.pkl')

In [18]:
data03

,date,qty,count,weekday,month,wti_price,day,is_weekend,quarter,is_quarter_start,...,category_min7,weekly_qty_sum,count_city,city_count_ma7,city_count_std7,FOMC,weekday_avg_qty,is_first_week,is_last_week,target
0,2014-01-02,9853.0,4422.0,Thursday,1,95.140000,2,0,1,0,...,10142.0,9853.0,45969.0,45969.000000,2533.563597,2.985,6315.245399,1,0,15153.0
1,2014-01-03,8647.0,4167.0,Friday,1,94.400000,3,0,1,0,...,8980.0,18500.0,42386.0,44177.500000,2533.563597,2.995,7675.257669,1,0,15606.0
2,2014-01-04,15153.0,5341.0,Saturday,1,94.400000,4,1,1,0,...,8980.0,33653.0,52293.0,46882.666667,5016.298669,2.995,11864.363636,1,0,7900.0
3,2014-01-05,15606.0,5123.0,Sunday,1,94.400000,5,1,1,0,...,8980.0,49259.0,49199.0,47461.750000,4256.389501,2.995,12851.171779,1,0,7188.0
4,2014-01-06,7900.0,3917.0,Monday,1,93.973333,6,0,1,0,...,8083.0,57159.0,39105.0,45790.400000,5249.256404,2.961,7802.484848,1,0,8800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,2017-02-24,10208.0,4549.0,Friday,2,53.577778,24,0,1,0,...,6143.0,67617.0,43170.0,41286.571429,4518.085098,2.317,7675.257669,0,0,10108.0
1144,2017-02-25,12361.0,4619.0,Saturday,2,53.577778,25,1,1,0,...,6143.0,63647.0,46801.0,40719.571429,3180.713749,2.317,11864.363636,0,1,10006.0
1145,2017-02-26,10108.0,3877.0,Sunday,2,53.577778,26,1,1,0,...,6143.0,61732.0,38062.0,40437.142857,3335.235795,2.317,12851.171779,0,1,14374.0
1146,2017-02-27,10006.0,3961.0,Monday,2,53.697778,27,0,1,0,...,7415.0,65734.0,38858.0,40542.571429,3260.611438,2.369,7802.484848,0,1,14374.0


# 2. 4차 모델링

- Random Forest, LigntGBM 알고리즘으로 모델링합니다.
- 이번에는 하이퍼파라미터 튜닝을 통해 모델의 성능을 높여봅니다.
- 이전 실습에서 만든 preproc 함수를 그대로 선언합니다.
- 변화되는 모델의 성능을 기록합니다.

In [20]:
def preproc(data):
    # x, y 분리
    y= data['target']
    x= data.drop(columns= ['date','target'])
    # 가변수화 : 가변수화 할 컬럼 =  weekday , month 
    dummy_cols = ['weekday','month']
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    x['month'] = pd.Categorical(x['month'],categories= months)
    x = pd.get_dummies(data=x, columns = dummy_cols , drop_first=True)

    # 학습용, 검증용 분리
    x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=120, shuffle =False)

    # 결과 반환
    return x_train, x_val, y_train, y_val


In [21]:
# x_train, x_val , y_train, y_val = preproc(data03)
# # 스케일링
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)


## (1) Random Forest 모델

### 1) 상품 3 모델링

- 모델 이름은 model03_rdf로 합니다.

In [24]:
# x_train, x_val , y_train, y_val = preproc(data03)
# # 스케일링
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)

In [25]:
# # 파라미터 선언
# param = {
#     'n_estimators': [100, 200, 300, 400, 500],  # 트리 개수
#     'max_depth': [3, 5, 7, 9, 11],              # 트리 깊이
#     'min_samples_split': [2, 5, 10],            # 노드 분할에 필요한 최소 샘플 수
#     'min_samples_leaf': [1, 2, 4],              # 리프 노드의 최소 샘플 수
#     'max_features': ['auto', 'sqrt', 'log2']    # 최대 사용할 특성 수
# }

# # Grid Search 선언
# model03_rdf = GridSearchCV(RandomForestRegressor(), param, cv=3)

# # Grid Search 실행
# model03_rdf.fit(x_train, y_train)

# # 최적의 파라미터 확인
# print("Best Parameters:", model03_rdf.best_params_)

# # 최적의 모델 성능 확인
# print("Best Score:", model03_rdf.best_score_)



### 2) 상품 7 모델링

- 모델 이름은 model07_rdf로 합니다.

In [27]:
# x_train, x_val , y_train, y_val = preproc(data07)
# # 스케일링
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)

In [28]:
# # 파라미터 선언
# param = {
#     'n_estimators': [50, 100,200],  # 트리 개수
#     'max_depth': range(1, 5)                       # 트리 깊이 1~10
# }

# # Grid Search 선언
# model07_rdf = GridSearchCV(RandomForestRegressor(), param, cv=3)

# # Grid Search 실행
# model07_rdf.fit(x_train, y_train)


# # 최적의 파라미터 확인
# print("Best Parameters:", model07_rdf.best_params_)

# # 최적의 모델 성능 확인
# print("Best Score:", model07_rdf.best_score_)

### 3) 상품 12 모델링

- 모델 이름은 model12_rdf로 합니다.

In [30]:
# x_train, x_val , y_train, y_val = preproc(data12)
# # 스케일링
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)

In [31]:
# # 파라미터 선언
# param = {
#     'n_estimators': [50, 100, 200],  # 트리 개수
#     'max_depth': range(1, 5)                       # 트리 깊이 1~10
# }

# # Grid Search 선언
# model12_rdf = GridSearchCV(RandomForestRegressor(), param, cv=3)

# # Grid Search 실행
# model12_rdf.fit(x_train, y_train)

# # 최적의 파라미터 확인
# print("Best Parameters:", model12_rdf.best_params_)

# # 최적의 모델 성능 확인
# print("Best Score:", model12_rdf.best_score_)

## (2) LightGBM 모델

### 1) 상품 3 모델링

- 모델 이름은 model03_lgb로 합니다.

In [34]:
# x_train, x_val , y_train, y_val = preproc(data03)
# # 스케일링
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)

In [35]:
# # 파라미터 범위 설정
# param = {
#     'n_estimators': [50, 100, 200],  # 트리 개수
#     'learning_rate': [0.01, 0.05, 0.1]    # 학습률
# }

# # Grid Search 선언
# model03_lgb = GridSearchCV(LGBMRegressor(verbose = False), param, cv=3)

# # Grid Search 실행
# model03_lgb.fit(x_train, y_train)

# # 최적의 파라미터 확인
# print("Best Parameters:", model03_lgb.best_params_)

# # 최적의 모델 성능 확인
# print("Best Score:", model03_lgb.best_score_)

### 2) 상품 7 모델링

- 모델 이름은 model07_lgb로 합니다.

In [37]:
# x_train, x_val , y_train, y_val = preproc(data07)
# # 스케일링
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)

In [38]:
# # 파라미터 범위 설정
# param = {
#     'n_estimators': [50, 100, 150, 200],  # 트리 개수
#     'learning_rate': [0.01, 0.05, 0.1]    # 학습률
# }

# # Grid Search 선언
# model07_lgb = GridSearchCV(LGBMRegressor( verbose = False), param, cv=3)

# # Grid Search 실행
# model07_lgb.fit(x_train, y_train)

# # 최적의 파라미터 확인
# print("Best Parameters:", model07_lgb.best_params_)

# # 최적의 모델 성능 확인
# print("Best Score:", model07_lgb.best_score_)

### 3) 상품 12 모델링

- 모델 이름은 model12_lgb로 합니다.

In [40]:
# x_train, x_val , y_train, y_val = preproc(data12)
# # 스케일링
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)

In [41]:
# # 파라미터 범위 설정
# param = {
#     'n_estimators': [50, 100, 200],  # 트리 개수
#     'learning_rate': [0.01, 0.05, 0.1]    # 학습률
# }

# # Grid Search 선언
# model12_lgb = GridSearchCV(LGBMRegressor(verbose = False), param, cv=3)

# # Grid Search 실행
# model12_lgb.fit(x_train, y_train)

# # 최적의 파라미터 확인
# print("Best Parameters:", model12_lgb.best_params_)

# # 최적의 모델 성능 확인
# print("Best Score:", model12_lgb.best_score_)

# 3. 파이프라인 구축

- 새로 읽어온 데이터에 대해 모델이 예측할 수 있는 형태의 데이터 셋을 만들어야 합니다.
- 이러한 데이터 셋을 만드는 파이프라인을 함수를 만들어 보세요.

## (1) 새로운 데이터 불러오기

- 평가에 사용할 새로운 데이터를 불러옵니다.
- 데이터프레임 이름은 다음과 같이 통일합니다.
    - sales: 판매 정보
    - orders: 고객 방문수
    - oil_price: 휘발유 가격
    - stores: 매장 정보
    - products: 상품 정보

In [44]:
# 데이터 불러오기
sales = pd.read_csv(path + 'sales_test.csv')
orders = pd.read_csv(path + 'orders_test.csv')
oil_price = pd.read_csv(path + 'oil_price_test.csv')
stores = pd.read_csv(path + 'stores.csv')
products = pd.read_csv(path + 'products.csv')

In [45]:
# datetime 형으로 변환
sales['date'] = pd.to_datetime(sales['date'] )
oil_price['date'] = pd.to_datetime(oil_price['date'] )
orders['date'] = pd.to_datetime(orders['date'] )

## (2) 데이터 셋 구성 함수

- 이전 실습에서 '데이터 셋 구성하기 #2' 에서 작성한 코드를 가져와 함수를 다시 선언합니다.
    - 함수 이름: make_dataset
- 참고: 모델링 과정부터 함수를 만들어 사용했기에, 함수를 다시 만들 필요가 없습니다.

In [47]:
!pip install yfinance

In [48]:
import yfinance as yf
# 미국 10년 국채 금리 데이터를 가져오는 예시 (US10Y)
finance = yf.download('^TNX', start='2014-01-01', end='2017-02-28')  # 미국 10년 국채 금리

finance = finance.reset_index()
# 미국 10년 국채 금리는 시장에서 중요한 경제적 신뢰도를 반영합니다. 
# 높은 금리는 일반적으로 경제 성장과 인플레이션 우려와 관련이 있고,  >> 높으면  경제성장
# 낮은 금리는 경기 침체나 경제 불확실성과 관련이 있습니다           >> 낮으면 경제 침체
# 이중에서 당일 종가 데이터만 가져오기에 Close 만 가져오자  (Adj Close 수정된 종가는 Close와 같음 )
# # 'Date' 컬럼에서 날짜만 추출하여 '년-월-일' 형식으로 변경
finance['Date'] = pd.to_datetime(finance['Date']).dt.strftime('%Y-%m-%d')

# # 'Close'와 'Date' 컬럼만 선택
bond_data = finance[['Date', 'Close']]
# ticket, price 인덱스 제거 
bond_data = bond_data.reset_index()

# 날짜를 인덱스로 설정하고 날짜 범위에 맞는 모든 날짜 생성
bond_data['Date'] = pd.to_datetime(bond_data['Date'])
date_range = pd.date_range(start='2014-01-02', end='2017-02-28')

# 모든 날짜에 대해 종가를 채우기 전에, 데이터를 날짜별로 정렬
data_sorted = bond_data.set_index('Date').reindex(date_range)

# 전날 종가로 결측치를 채움
data_sorted['Close'] = data_sorted['Close'].fillna(method='ffill')

# 인덱스를 컬럼으로 다시 리셋
data_final = data_sorted.reset_index()
# 불필요한 컬럼 제거: 'Price', 'level_0', 'index', 'Ticker' 등을 제거
data_final_cleaned = data_final.drop(columns=['index'])

# 컬럼명 변경
data_final_cleaned.rename(columns={'level_0': 'date'}, inplace=True)
data_final_cleaned.columns = ['date', 'FOMC']
# 결과 확인
interest_rate = data_final_cleaned.copy()
interest_rate

[*********************100%***********************]  1 of 1 completed


,date,FOMC
0,2014-01-02,2.985
1,2014-01-03,2.995
2,2014-01-04,2.995
3,2014-01-05,2.995
4,2014-01-06,2.961
...,...,...
1149,2017-02-24,2.317
1150,2017-02-25,2.317
1151,2017-02-26,2.317
1152,2017-02-27,2.369


In [49]:
# 함수 만들기(수정 필요) (김명제님 )
def make_dataset(store_id, product_id):
    # 데이터 준비
    leadtime = products.loc[products['product_id']==product_id, 'leadtime'].values[0]
    temp1 = sales.loc[(sales['store_id']==store_id) & (sales['product_id']==product_id), ['date', 'qty']]
    temp2 = orders.loc[orders['store_id']==store_id, ['date', 'count']]
    temp3 = pd.merge(temp1, temp2, on='date', how='left')
 
    # 날짜 요소 추출
    temp3['weekday'] = temp3['date'].dt.day_name()
    temp3['month'] = temp3['date'].dt.month
 
    # Oil Price
    temp3 = pd.merge(temp3, oil_price, on='date', how='left')
    temp3['wti_price'] = temp3['wti_price'].rolling(14, min_periods=1).mean()
 
    # 추가(날짜)
    temp3['day'] = temp3['date'].dt.day
    temp3['is_weekend'] = temp3['weekday'].isin(['Saturday', 'Sunday']).astype(int)
    temp3['quarter'] = temp3['date'].dt.quarter
    temp3['is_quarter_start'] = temp3['date'].dt.is_quarter_start.astype(int)
    temp3['is_quarter_end'] = temp3['date'].dt.is_quarter_end.astype(int)
    temp3['week'] = temp3['date'].dt.isocalendar().week
    temp3['is_month_start'] = temp3['date'].dt.is_month_start.astype(int)
    temp3['is_month_end'] = temp3['date'].dt.is_month_end.astype(int)
    temp3['is_year_start'] = temp3['date'].dt.is_year_start.astype(int)
    temp3['is_year_end'] = temp3['date'].dt.is_year_end.astype(int)
 
    # 추가(qty lag)
    temp3['qty_lag_1'] = temp3['qty'].shift(1)
    temp3['qty_lag_2'] = temp3['qty'].shift(2)
    temp3['qty_lag_3'] = temp3['qty'].shift(3)
    temp3['qty_lag_4'] = temp3['qty'].shift(4)
    temp3['qty_lag_5'] = temp3['qty'].shift(5)
    temp3['qty_lag_6'] = temp3['qty'].shift(6)
    temp3['qty_lag_7'] = temp3['qty'].shift(7)
    temp3['qty_lag_14'] = temp3['qty'].shift(14)
 
    # 추가(방문수)
    temp3['count_mean3'] = temp3['count'].rolling(window=3, min_periods=1).mean()
    temp3['count_mean7'] = temp3['count'].rolling(window=7, min_periods=1).mean()
    temp3['count_mean14'] = temp3['count'].rolling(window=14, min_periods=1).mean()
    temp3['qty_per_customer'] = temp3['qty'] / temp3['count']
    temp3['count_std7'] = temp3['count'].rolling(window=7, min_periods=1).std()
 
    # 판매량
    temp3['qty_max7'] = temp3['qty'].rolling(window=7, min_periods=1).max()
    temp3['qty_min7'] = temp3['qty'].rolling(window=7, min_periods=1).min()
    temp3['qty_mean7'] = temp3['qty'].rolling(window=7, min_periods=1).mean()
    # 카테고리
    category = products.loc[products['product_id']==product_id, 'category'].values[0]
    category_products = products.loc[products['category']==category, 'product_id'].values
    category_sales = sales.loc[(sales['store_id']==store_id) &
                             (sales['product_id'].isin(category_products)),
                             ['date', 'qty']].groupby('date')['qty'].sum().reset_index()
    category_sales.columns = ['date', 'qty_category']
    temp3 = pd.merge(temp3, category_sales, on='date', how='left')
    temp3['category_mean7'] = temp3['qty_category'].rolling(window=7, min_periods=1).mean()
    temp3['category_min7'] = temp3['qty_category'].rolling(window=7, min_periods=1).min()

    # 주간 누적 판매량 (추가)
    temp3['weekly_qty_sum'] = temp3['qty'].rolling(7, min_periods=1).sum()
    # 도시
    city = stores.loc[stores['store_id']==store_id, 'city'].values[0]
    city_stores = stores.loc[stores['city']==city, 'store_id'].values
    city_customers = orders.loc[orders['store_id'].isin(city_stores),
                              ['date', 'count']].groupby('date')['count'].sum().reset_index()
    temp3 = pd.merge(temp3, city_customers, on='date', how='left', suffixes=('', '_city'))
    temp3['city_count_ma7'] = temp3['count_city'].rolling(window=7, min_periods=1).mean()
    temp3['city_count_std7'] = temp3['count_city'].rolling(window=7, min_periods=1).std()


    # (추가) 미국 당일 금리를 추가 
    temp3 = pd.merge(temp3, interest_rate, on='date', how='left') 
    # (추가) (요일별 평균)
    temp3['weekday_avg_qty'] = temp3.groupby('weekday')['qty'].transform('mean')
    #(추가) 월초 , 월밀 = 월급날 > 소비증가 > 매출증가 
    temp3['is_first_week'] = (temp3['date'].dt.day <= 7).astype(int)
    temp3['is_last_week'] = (temp3['date'].dt.day >= 25).astype(int)

    
    # Target 추가
    temp3['target'] = temp3['qty'].shift(-leadtime)
 
    # 결측치 처리
    temp3.interpolate(method='linear', inplace=True)
    temp3.fillna(method='bfill', inplace=True)
 
    # 결과 반환
    return temp3

## (3) 데이터 분리 함수

- 새로운 데이터를 받아서 x, y로 분리해 반환하는 함수를 만듭니다.
    - 함수 이름: preproc_newdata
    - 입력: 새로운 데이터(data)
    - 처리: x와 y 분리, x 가변수화
- 참고: 이전에 만든 preproc 함수는 학습용, 검증용을 분리하므로 지금 단계에서는 사용할 수 없습니다.
- 참고: 하지만 대부분의 코드는 preproc 함수와 같으니, preproc 함수 코드를 참고하시 바랍니다.

In [51]:
def preproc_newdata(data):  # x_train , y_train 을 반환
    # x, y 분리
    y= data['target']
    x= data.drop(columns= ['date','target'])
    # 가변수화 : 가변수화 할 컬럼 =  weekday , month 
    dummy_cols = ['weekday','month']
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    x['month'] = pd.Categorical(x['month'],categories= months)
    x = pd.get_dummies(data=x, columns = dummy_cols , drop_first=True)
    # # 학습용, 검증용 분리
    # x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=120, shuffle =False)
    ...

    return x, y

## (4) 데이터 파이프라인

- 위 두 함수를 사용해 지정한 매장, 지정한 상품의 데이터를 전처리해 x와 y를 반환하는 함수를 만듭니다.
    - 함수 이름: data_pipeline
    - 입력: 매장번호(store_id), 상품번호(product_id)

In [53]:
def data_pipeline(store_id, product_id): # 테스트용 데이터 생성
    temp=make_dataset(store_id, product_id)
    x,y = preproc_newdata(temp)
    
    return x, y

- 위 함수는 이후에 다음과 같은 형태로 사용될 것입니다.

~~~
x_test, y_test = data_pipeline(매장번호, 상품번호)
y_pred = model.predict(x_test)
~~~

# 4. 최종 평가

- 새로운 데이터에 대해 예측하고 성능을 평가합니다.
- 재고금액을 평가하는 다음 함수를 활용해 비즈니스 관점의 평가를 수행하세요.
- 최종 평가 결과를 기록하고 비교합니다.

In [56]:
# 재고량 평가 함수 만들기
def inv_simulator(y, pred, safe_stock, price):

    # 시뮬레이션 df 틀 만들기
    temp = pd.DataFrame({'y': y.reshape(-1,), 'pred': pred.reshape(-1,).round()})
    temp['y'] = temp['y'].astype(int)
    temp['pred'] = temp['pred'].astype(int)
    temp['safe_stock'] = safe_stock

    temp['order'] = 0
    temp['receive'] = 0
    temp['base_stock'] = 0
    temp['close_stock'] = 0

    # 시뮬레이션
    for i in range(len(temp)-2):  # 발주량은 2일 후 판매 예측량에 기초하므로 계산을 위해 마지막 2개 행 제외
        if i == 0:  #첫 행, 2일 전 데이터가 없으므로
            temp.loc[i, 'receive'] = temp.loc[i, 'y']  # 입고량은 실판매량으로 계산
            temp.loc[i, 'base_stock'] = temp.loc[i, 'receive'] + safe_stock  # 기초재고는 실판매량 + 안전재고로 계산

        elif i == 1: # 2일 전 행, 2일 전 데이터가 없음
            temp.loc[i, 'receive'] = temp.loc[i, 'y'] # 입고량은 실판매량으로 계산
            temp.loc[i, 'base_stock'] = temp.loc[i,'receive'] + temp.loc[i-1, 'close_stock']
        else:        # 나머지 전체 행
            temp.loc[i, 'receive'] = temp.loc[i-2,'order']    # 입고량 = 2일 전 발주량
            temp.loc[i, 'base_stock'] = temp.loc[i, 'receive']+temp.loc[i-1, 'close_stock']  # 기초재고 = 입고량 + 전날 기말재고

        # 기말재고 = 기초재고 - 판매량, 만약 0보다 작으면 0으로
        stock = round(temp.loc[i, 'base_stock'] - temp.loc[i, 'y'])
        temp.loc[i, 'close_stock'] = np.where(stock > 0, stock, 0)

        # 발주량 = 2일후 판매예측량 + 안전재고 - 기말재고,  만약 주문량이 0보다 작으면 0
        order = temp.loc[i+2, 'pred'] + safe_stock - temp.loc[i, 'close_stock']
        temp.loc[i, 'order'] = np.where(order>0, order, 0)

    # 기회손실 = 만약 (기초재고 - 실판매량)이 0보다 작으면, 그만큼이 기회손실
    temp['lost'] = np.where((temp['base_stock']-temp['y']) < 0, (temp['base_stock']-temp['y']), 0).round()

    inventory = temp[:len(temp) - 2]

    # 측정지표 계산
    DailyStock = ((inventory['base_stock'] + inventory['close_stock']) / 2)

    AvgDailyStock = round(DailyStock.mean(), 3)
    AvgDailyStockAmt = AvgDailyStock * price * 0.5
    lost_sum = inventory['lost'].sum()

    print(f'* 일평균 재고량: {AvgDailyStock:.2f}')
    print(f'* 일평균 재고금액: {AvgDailyStockAmt:.2f}')
    print(f'* 기회손실 수량: {lost_sum}')

    return inventory

## (1) 상품 3

- 3번 상품에 대해 모델 성능을 평가하고 기록합니다.

### 1) Random Forest 모델

- Random Forest 모델로 예측하고 평가합니다.

In [364]:
x_train, x_val , y_train, y_val = preproc(data03)
# 스케일링
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [366]:
# 예측, 평가하기
x_test, y_test = data_pipeline(44, 3)
# x_train 스케일러 대로 x_test도 스케일링
x_test = scaler.transform(x_test)

model03_rdf = RandomForestRegressor(max_depth=9, 
                                    max_features='sqrt',
                                    min_samples_leaf = 2,
                                    min_samples_split= 10,
                                    n_estimators= 200)

model03_rdf.fit(x_train, y_train)

y_pred = model03_rdf.predict(x_test)

# 평가
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MAPE:", f"{mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}%")
print("R2-Score:", r2_score(y_test, y_pred))


RMSE: 3307.5076637162015
MAE: 2067.5590039495246
MAPE: 14.48%
R2-Score: 0.3353067979457469


In [382]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred,24526, 8)

# 일평균 재고금액이 가장 적게 나오도록 
# 기회손실 수량 == 0

* 일평균 재고량: 31583.65
* 일평균 재고금액: 126334.62
* 기회손실 수량: 0


### 2) LightGBM 모델

- LightGBM 모델로 예측하고 평가합니다.

In [66]:
x_train, x_val , y_train, y_val = preproc(data03)
# 스케일링
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [67]:
# 예측, 평가하기
x_test, y_test = data_pipeline(44, 3)
# x_train 스케일러 대로 x_test도 스케일링
x_test = scaler.transform(x_test)

model03_lgb = LGBMRegressor(learning_rate=0.05, n_estimators= 200,verbose= -1)

model03_lgb.fit(x_train, y_train)

y_pred = model03_lgb.predict(x_test)

# 평가
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MAPE:", f"{mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}%")
print("R2-Score:", r2_score(y_test, y_pred))

RMSE: 4503.987197041826
MAE: 2957.0896162772274
MAPE: 21.05%
R2-Score: -0.23257630194242274


In [68]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred,30000, 15)


* 일평균 재고량: 37530.03
* 일평균 재고금액: 281475.26
* 기회손실 수량: 0


## (2) 상품 7

- 7번 상품에 대해 모델 성능을 평가하고 기록합니다.

### 1) Random Forest 모델

- Random Forest 모델로 예측하고 평가합니다.

In [288]:
x_train, x_val , y_train, y_val = preproc(data07)
# 스케일링
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [290]:
# 예측, 평가하기
# 예측, 평가하기
x_test, y_test = data_pipeline(44, 7)
# x_train 스케일러 대로 x_test도 스케일링
x_test = scaler.transform(x_test)

model07_rdf = RandomForestRegressor(max_depth=9, 
                                    max_features='sqrt',
                                    min_samples_leaf = 2,
                                    min_samples_split= 10,
                                    n_estimators= 200)

model07_rdf.fit(x_train, y_train)

y_pred = model07_rdf.predict(x_test)

# 평가
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MAPE:", f"{mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}%")
print("R2-Score:", r2_score(y_test, y_pred))


RMSE: 495.7878484924969
MAE: 390.2662909951066
MAPE: 15.24%
R2-Score: 0.49429585330948267


In [360]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred,3173, 15)


* 일평균 재고량: 4669.52
* 일평균 재고금액: 35021.38
* 기회손실 수량: 0


### 2) LightGBM 모델

- LightGBM 모델로 예측하고 평가합니다.

In [75]:
# 예측, 평가하기
x_train, x_val , y_train, y_val = preproc(data07)
# 스케일링
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)


In [76]:
# 비즈니스 평가
# 예측, 평가하기
x_test, y_test = data_pipeline(44, 7)
# x_train 스케일러 대로 x_test도 스케일링
x_test = scaler.transform(x_test)

model07_lgb = LGBMRegressor(learning_rate=0.05, n_estimators= 200,verbose= -1)

model07_lgb.fit(x_train, y_train)

y_pred = model03_lgb.predict(x_test)

# 평가
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MAPE:", f"{mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}%")
print("R2-Score:", r2_score(y_test, y_pred))



RMSE: 6690.8626044832845
MAE: 6389.4941263448545
MAPE: 251.37%
R2-Score: -91.10198323633477


In [77]:
result = inv_simulator(y_test.values, y_pred,30000, 15)

* 일평균 재고량: 37566.24
* 일평균 재고금액: 281746.81
* 기회손실 수량: 0


## (3) 상품 12

- 12번 상품에 대해 모델 성능을 평가하고 기록합니다.

### 1) Random Forest 모델

- Random Forest 모델로 예측하고 평가합니다.

In [272]:
x_train, x_val , y_train, y_val = preproc(data12)
# 스케일링
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [274]:
# 예측, 평가하기
x_test, y_test = data_pipeline(44, 12)
# x_train 스케일러 대로 x_test도 스케일링
x_test = scaler.transform(x_test)

model12_rdf = RandomForestRegressor(max_depth=9, 
                                    max_features='sqrt',
                                    min_samples_leaf = 2,
                                    min_samples_split= 10,
                                    n_estimators= 200)

model12_rdf.fit(x_train, y_train)

y_pred = model12_rdf.predict(x_test)

# 평가
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MAPE:", f"{mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}%")
print("R2-Score:", r2_score(y_test, y_pred))


RMSE: 2600.7577172963925
MAE: 1996.6906925876754
MAPE: 17.78%
R2-Score: 0.414808055847086


In [275]:
# 비즈니스 평가

result = inv_simulator(y_test.values, y_pred,17721, 6)
result

* 일평균 재고량: 25112.26
* 일평균 재고금액: 75336.78
* 기회손실 수량: 0


,y,pred,safe_stock,order,receive,base_stock,close_stock,lost
0,11207,12347,17721,11946,11207,28928,17721,0
1,16590,13169,17721,11133,16590,34311,17721,0
2,18768,11946,17721,19365,11946,29667,10899,0
3,8985,11133,17721,17056,11133,22032,13047,0
4,8678,12543,17721,3610,19365,32412,23734,0
5,8965,12382,17721,0,17056,40790,31825,0
6,8208,9623,17721,4174,3610,35435,27227,0
7,9083,10538,17721,11443,0,27227,18144,0
8,14392,13680,17721,19460,4174,22318,7926,0
9,16478,11866,17721,24035,11443,19369,2891,0


In [384]:
plot_model_result(y_train ,y_val, y_pred)

ValueError: Length of values (31) does not match length of index (120)

### 2) LightGBM 모델

- LightGBM 모델로 예측하고 평가합니다.

In [84]:

x_train, x_val , y_train, y_val = preproc(data12)
# 스케일링
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)


In [85]:

# 예측, 평가하기
x_test, y_test = data_pipeline(44, 12)
# x_train 스케일러 대로 x_test도 스케일링
x_test = scaler.transform(x_test)

model07_lgb = LGBMRegressor(learning_rate=0.05, n_estimators= 200,verbose= -1)

model07_lgb.fit(x_train, y_train)

y_pred = model03_lgb.predict(x_test)

# 평가
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MAPE:", f"{mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}%")
print("R2-Score:", r2_score(y_test, y_pred))



RMSE: 3128.027890073941
MAE: 2097.918914487855
MAPE: 17.30%
R2-Score: 0.15347501541959463


In [86]:
result = inv_simulator(y_test.values, y_pred,30000, 15)

* 일평균 재고량: 35358.85
* 일평균 재고금액: 265191.34
* 기회손실 수량: 0
